# <h1><center> **Hackathon AI - VU Amsterdam** </center></h1>
<h2><center>Physiological sensor data </center></h2>

<br>
<center><img src="Sensor.png" alt="" width="700"/></center>
<br>

In this working group, your group are going to explore the ways and methods one can extract sensible knowledge from sensory data. Also, you will acquire some first insight into modern machine learning algorithms. The goal is (just as overall with the Hackathon) to have fun, but also to learn new AI skills! So let's get started!

In the first step, you should download the dataset. The dataset can be downloaded [here](URL einfuegen). 


## Train and test split

Normally, you should split your dataset into a so-called training dataset and a test set. Yet, in this workshop you will work with a dataset and generate your own test set! That is, you will first try to come up with a 'good'model and then collect your data and then see and check the performance of the algorithm again!

## Getting the app for gathering your own data

The app that we will use to gather your own data is 'Sensor Logger' developed by Dr. Kelvin Choi. The documentation as well as the links to the playstore can be found [here](https://www.tszheichoi.com/sensorlogger)

Some people can get acquainted with the app and also start to collect the test data. The more data you have the better! Also, make sure to collect data for the following features:


**Accelerometer**:
The acceleration of the device in the x,y,z axis (so basically acceleration in space). 

**Gyroscope**:
The angular speed of the device, again in the x,y,z axis.

**Magnetometer**:
The magnetometer value of the device.

**Light lux feature**

### Basic Terminologies

Before we start with exploring the dataset, you should get familiar with a few basic terminologies. These definitions were taken from the book '“Machine Learning for the Quantified Self – On the Art of Learning from Sensory Data” (2018) by Hoogendoorn and Funk. 

**Machine learning**:

Machine learning is the task to automatically identify patterns from the data.

In machine learning, the difference between *supervised learning* and *unsupervised learning* is very important. I am sure that you have already heard these two terms, but here is a quick refresher:

**Supervised learning**:

Supervised learning relates to the task of inferring a function from labelled data. For example, one wants to predict the blood glucose level based on the food a person has eaten. 

**Unsupervised learning**:

On the contrary to supervised learning, unsupervised learning wants to find patterns and associations among the attributes (variables, features). For example, one wants to find different clusters of customers based on their consumer behavior (past purchases etc.).

In the following workshop, we will be mainly concerned with supervised learning. 

### Importing important libraries 

In [ ]:
import pandas as pd
import io
import numpy as np
import datetime
import pandas as pd
import re
import copy
from pathlib import Path
import copy
import os
import sys
from datetime import datetime, timedelta
import matplotlib.pyplot as plot
import matplotlib.dates as md
import scipy.stats
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.mixture import GaussianMixture
import seaborn as sns
from sklearn.pipeline import Pipeline
from statsmodels.graphics.gofplots import qqplot
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import BayesianRidge
from sklearn.compose import make_column_transformer, ColumnTransformer
#from category_encoders import *
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import BayesianRidge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
#from category_encoders.target_encoder import TargetEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from imblearn.over_sampling import SMOTE
import numpy as np
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import f1_score
#from prince import PCA
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectKBest, f_classif
from scipy.stats import skew 
from sklearn.metrics import mean_squared_error, make_scorer 
#import shap
from sklearn.model_selection import train_test_split
#from mlens.ensemble import SuperLearner, BlendEnsemble, Subsemble, SequentialEnsemble
from sklearn.cluster import KMeans
import os
from google.colab import files
from datetime import datetime, timedelta
from pytz import timezone
from sklearn.neighbors import LocalOutlierFactor
from sklearn.ensemble import AdaBoostClassifier
from scipy.stats import shapiro

# Part 1: Preprocessing of the data

As you can see, there are several data files available for you to use in the component workshop. The most important data files are the **Accelerometer, Gyroscope and the Magnetometer**. There are many more features (variables) that one could potentially use, but given the time constraint, we want to focus only on those simple ones.

**Accelerometer**:
The acceleration of the device in the x,y,z axis (so basically acceleration in space). 

**Gyroscope**:
The angular speed of the device, again in the x,y,z axis.

**Magnetometer**:
The magnetometer value of the device.

**Light lux feature**

Furthermore, we will use the light feature as well!

It is important to check what your current working directory is



In [ ]:
os.getcwd()

Sensor data usually come with a timestamp expressed in nano seconds. Hence, one needs to convert the timestamp into a format that is readable for us humans. Luckily, this part has been already done for you, so you can focus on more fun parts! Yet, below you can find the code that was used to convert the timestamps into human-readable formats.

In [ ]:
def format_my_nanos_features(nanos):
    dt = datetime.fromtimestamp(nanos['timestamps'] / 1e9)
    dt = dt.astimezone(timezone('US/Pacific'))
    print(dt)
    return '{}{:03.0f}'.format(dt.strftime('%Y/%m/%d %H:%M:%S.%f'), nanos['timestamps'] % 1e3)

def format_my_nanos_label_start(nanos):
    dt = datetime.fromtimestamp(nanos['label_start']/ 1e9)
    dt = dt.astimezone(timezone('US/Pacific'))
    print(dt)
    return '{}{:03.0f}'.format(dt.strftime('%Y/%m/%d %H:%M:%S.%f'), nanos['label_start'] % 1e3)

def format_my_nanos_label_end(nanos):
    dt = datetime.fromtimestamp(nanos['label_end']/ 1e9)
    dt = dt.astimezone(timezone('US/Pacific'))
    print(dt)
    return '{}{:03.0f}'.format(dt.strftime('%Y/%m/%d %H:%M:%S.%f'), nanos['label_end'] % 1e3)


## Converting raw data to an aggregated data format 

An important aspect is the timestep of the data that we going to use. For instance, we could say that we want to measure the above-mentioned features 4 times per second, i.e. 250 nano seconds. Alternatively, we could say we would like to have measure only every half a minute. 

**Question** 

What are advantages and disadvantages of measuring at a higher frequency, i.e. 250 nano seconds?

*Here you can write some thoughts*

After we have decided on the specific measure frequency or also called **granularity**, we need to decide how we deal with all observations that fall into the time window. For istance, what if we have 10 observations within the decided time window of 30 seconds? How do we include the information of all those ten observations in one single observation?

In practice, people use different methods based on the specific variable one considers. As we deal with numerical variables, i.e. it takes real values, most people simply take the mean. In the above-mentioned example, one would only use a simple average of all 10 observations. 

For the label data, things are different since we are dealing with a categorical variable. That is, the value of the variables are categories, 'sitting' or 'on the table'. For this, one simply creates a new variable which takes the value of 1 if a specific label is present during the time window, and 0 otherwise. Such a variable is called a **binary** variable as it takes only two values.

As the preprocessing and aggregation takes a lot of time, and since time is restricted in this Hackathon, this part has already been done for you. Yet, it is important to know how your data has been preprocessed. 

The following datasets have a granularity of 250 nanoseconds. This is the granularity with which we will work with.

**Important** 

Do not change the code below!

In [ ]:
class CreateDataset:

    base_dir = ''
    granularity = 0
    data_table = None
    #file = files.upload()

    def __init__(self, base_dir, granularity):
        self.base_dir = base_dir
        self.granularity = granularity    
    
    # Create an initial data table with entries from start till end time, with steps
    # of size granularity. Granularity is specified in milliseconds
    def create_timestamps(self, start_time, end_time):
        return pd.date_range(start_time, end_time, freq=str(self.granularity)+'ms')

    def create_dataset(self, start_time, end_time, cols, prefix):
        c = copy.deepcopy(cols)
        if not prefix == '':
            for i in range(0, len(c)):
                c[i] = str(prefix) + str(c[i])
        timestamps = self.create_timestamps(start_time, end_time)
        self.data_table = pd.DataFrame(index=timestamps, columns=c)

    # Add numerical data, we assume timestamps in the form of nanoseconds from the epoch
    def add_numerical_dataset(self, file, timestamp_col, value_cols, aggregation='avg', prefix=''):
        print(f'Reading data from {file}')
        #dataset = pd.read_csv(self.base_dir / file, skipinitialspace=True)
        dataset = pd.read_csv(file)
        #print(f'Reading data from')
        #dataset = pd.read_csv(io.BytesIO(uploaded[file]))
        #dataset = pd.read_csv(self.base_dir / file, skipinitialspace=True)

        # Convert timestamps to dates
        dataset[timestamp_col] = pd.to_datetime(dataset[timestamp_col])

        # Create a table based on the times found in the dataset
        if self.data_table is None:
            self.create_dataset(min(dataset[timestamp_col]), max(dataset[timestamp_col]), value_cols, prefix)
        else:
            for col in value_cols:
                self.data_table[str(prefix) + str(col)] = np.nan

        # Over all rows in the new table
        for i in range(0, len(self.data_table.index)):
            # Select the relevant measurements.
            relevant_rows = dataset[
                (dataset[timestamp_col] >= self.data_table.index[i]) &
                (dataset[timestamp_col] < (self.data_table.index[i] +
                                           timedelta(milliseconds=self.granularity)))
            ]
            for col in value_cols:
                # Take the average value
                if len(relevant_rows) > 0:
                    if aggregation == 'avg':
                        self.data_table.loc[self.data_table.index[i], str(prefix)+str(col)] = np.average(relevant_rows[col])
                    else:
                        raise ValueError(f"Unknown aggregation {aggregation}")
                else:
                    self.data_table.loc[self.data_table.index[i], str(prefix)+str(col)] = np.nan

    # Remove undesired value from the names.
    def clean_name(self, name):
        return re.sub('[^0-9a-zA-Z]+', '', name)

    # Add data in which we have rows that indicate the occurrence of a certain event with a given start and end time.
    # 'aggregation' can be 'sum' or 'binary'.
    def add_event_dataset(self, file, start_timestamp_col, end_timestamp_col, value_col, aggregation='sum'):
        print(f'Reading data from {file}')
        dataset = pd.read_csv(self.base_dir / file)

        # Convert timestamps to datetime.
        dataset[start_timestamp_col] = pd.to_datetime(dataset[start_timestamp_col])
        dataset[end_timestamp_col] = pd.to_datetime(dataset[end_timestamp_col])

        # Clean the event values in the dataset
        dataset[value_col] = dataset[value_col].apply(self.clean_name)
        event_values = dataset[value_col].unique()

        # Add columns for all possible values (or create a new dataset if empty), set the default to 0 occurrences
        if self.data_table is None:
            self.create_dataset(min(dataset[start_timestamp_col]), max(dataset[end_timestamp_col]), event_values, value_col)
        for col in event_values:
            self.data_table[(str(value_col) + str(col))] = 0

        # Now we need to start counting by passing along the rows....
        for i in range(0, len(dataset.index)):
            # identify the time points of the row in our dataset and the value
            start = dataset[start_timestamp_col][i]
            end = dataset[end_timestamp_col][i]
            value = dataset[value_col][i]
            border = (start - timedelta(milliseconds=self.granularity))

            # get the right rows from our data table
            relevant_rows = self.data_table[(start <= (self.data_table.index +timedelta(milliseconds=self.granularity))) & (end > self.data_table.index)]

            # and add 1 to the rows if we take the sum
            if aggregation == 'sum':
                self.data_table.loc[relevant_rows.index, str(value_col) + str(value)] += 1
            # or set to 1 if we just want to know it happened
            elif aggregation == 'binary':
                self.data_table.loc[relevant_rows.index, str(value_col) + str(value)] = 1
            else:
                raise ValueError("Unknown aggregation '" + aggregation + "'")

    # This function returns the column names that have one of the strings expressed by 'ids' in the column name.
    def get_relevant_columns(self, ids):
        relevant_dataset_cols = []
        cols = list(self.data_table.columns)

        for id in ids:
            relevant_dataset_cols.extend([col for col in cols if id in col])

        return relevant_dataset_cols


## Binary classification 

As you can see, the task is to correctly predict if a person is 'active' or not. For this purpose, the data was also preprocessed in the sense that all activities that were either 'running' or 'walking' were classified as active, whereas all other labels such as 'Sitting', 'WashingHands', 'Standing', 'Driving', 'Eating' were classified as inactive. A value of 1 for the feature was assigned if the user was active, and zero otherwise.


## Importing the preprocessed data 

Make sure that you saved the data correctly and that you also refer to the right working directory.

In [ ]:
# If you work on a local machine, i.e. your laptop etc
#data_preprocessed250 = pd.read_csv('./intermediate_datafiles/data_250ns_no_time.csv', index_col=[0])

from google.colab import files
uploaded = files.upload()

#Now we need to make sure that you can convert it into a pandas dataframe!

data_preprocessed250 = pd.read_csv(io.BytesIO(uploaded['data_250ns_no_time.csv']))
#Do not change this line of code. It removes an unnecessary column!
data_preprocessed250 = data_preprocessed250.drop([data_preprocessed250.columns[0]], axis = 1)


In [ ]:
data_preprocessed250.head()

## Familiarize yourself with the data set 

The very first exercise before moving on to more advanced step in a machine learning pipeline is to familiarize yourself with the dataset and to get an overview of your data! A nice way is the .describe() method that gives you a very first overview of the data.

In [ ]:
data_preprocessed250.describe()

This gives you a nice overview of some descriptive statistics of the dataset. The 25%, 50% and 75% represent the 25 percent, median and the 75% quartile. 


A nice way to visualize this is by means of a boxplot. Below you can find some code to run a boxplot. 

In [ ]:
red_circle = dict(markerfacecolor = 'red', marker = 'o')
list_of_features_displayed = ['acc_phone_x', 'acc_phone_y','acc_phone_z']#You can add some more here if you want
dataset = data_preprocessed250

def plotting_boxplot(list_of_features_displayed, dataset):
    
    fig, axs = plt.subplots(1, len(list_of_features_displayed), figsize = (10,5))
    for i, ax in enumerate(axs.flat):
        list_of_features_displayed[i]
        ax.boxplot(dataset[list_of_features_displayed[i]], flierprops = red_circle)
        ax.set_title(list_of_features_displayed[i], fontsize = 10, fontweight = 'bold')
        ax.tick_params(axis = 'y', labelsize = 14)
    
    return plt.tight_layout()


In [ ]:
#Now you can plot and customize your plots!

plotting_boxplot(list_of_features_displayed, dataset)

## Handling outliers and missing data

An important step when preprocessing and preparing your data for further analysis is how remove outliers and also to handle missing data. According to Grubbs (1969), outliers are defined as follows:

**Outlier**: An outlier is an observation point that is distant from other observations.

As you might think, the definition what is distant is crucial for detecting outliers. Hence, this is something that you need to decide on. Also, there are plenty of different approaches that one can could consider for detecting outliers. In prinicple, there are two different roads/streams when it comes to outlier detection:

- distance based measures (use distance as a tool decide whether an obersation is an outlier or not)
- distribution based measures (assume an underlying distribution for the data generation process based on which an outlier is detected)





For both approaches, it is important to have scaled the dataset, i.e. it removes for each variable the mean and scales it to have unit variance. **Why do you think this is important?** 

The function below will scale all numerical data (therefore, the labeling data will not get scaled)

In [ ]:
class standardizer:
    def __init__(self, data):
        self.data = data
        self.a = [col for col in self.data.columns if not col.startswith('Active')]
        self.b = [col for col in self.data.columns if col.startswith('Active')]
        self.column_names = self.a + self.b
    
    def fit(self):
        self.ct = ColumnTransformer([
        ('scaling_numerical', StandardScaler(), [col for col in self.data.columns if not col.startswith('Active')])
        ], remainder='passthrough')
        #return self.data
        self.ct.fit(self.data)
        return self.ct
        print('The column transformer is fit now')
    
    def get_col(self):
        return self.column_names

In [ ]:
st = standardizer(data_preprocessed250)
my_standardizer= st.fit()
col_names = st.get_col()

#New data frame

data_scaled_250 = pd.DataFrame(my_standardizer.transform(data_preprocessed250), columns = col_names)

In [ ]:
data_scaled_250.head()

**Exercise**

Inspect the data and see the changes compared to the previous data set. Use for this again the 'dataset_name'.describe() and 'dataset_name'.summary() method!

Before moving on to remove outliers, it is also important to handle missing data. That is, for some observation there is no data available. This might result from technical issues, or even from some laziness from the person recording the data (for example, I forgot to label my data as 'On the Table'). In any way, we need find methods to impute missing values. Again, and as often times in machine learning, there are many options from which people can choose from. As always, it is important to consider the options and choose based upon your data and your knowledge!

There are 'simple' options for numerical and categorical data to choose from:
Replace missing values with 
- mean
- median

**Question** What is a main advantage of the median compared to the mean for imputing the data? How could you check that (**Tip, you have already seen a way to see if a median might be more appropriate**).

For categorical data people often use the mode (the most frequent value).

Alternatively, one could use more advanced approaches, such as parametric approaches (fitting a model such as linear regression). Do not worry if you do not know all of these, it is only for you important to know that there are alternativ ways to impute missing data.

The following function visualizes the missing value for the dataset!

In [ ]:
def visualize_missing_values(dataset):
    nulls = dataset.isnull().sum()
    sns.set(style="whitegrid")
    ax = sns.barplot(x=nulls.index, y=nulls.values/len(dataset))
    #ax = ax.set(label='common xlabel', ylabel='common ylabel')
    ax= ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
    return ax

In [ ]:
visualize_missing_values(data_scaled_250)

For preparing the dataset, write here the list for numerical and categorical data!

In [ ]:
#Write here the list of the features!# Alternative you can also try to code it :)
numbers = ['acc_phone_x', ... ]
categ = ['...']

In [ ]:
class Missing_Values:
    def __init__(self, dataset, method = 'mean'):
        self.method = method
        self.dataset = dataset
        self.numbers = [col for col in self.dataset.columns if not col.startswith('Active')]
        self.categ = [col for col in self.dataset.columns if col.startswith('Active')]
        self.labels = self.numbers + self.categ
        print(self.labels)
        
    def fit_imputer(self):
        #Build a pipeline for each type of data, one for numeric and one for categorical data
        categ_transformer = Pipeline(steps = [
            ('imputer', SimpleImputer(strategy = 'most_frequent'))
        ])

        number_transformer = Pipeline(steps=[
            ('imputer', SimpleImputer(strategy = self.method)) 
        ])

        #Next step: Make the column transformer:
        preprocessor = ColumnTransformer(
            transformers = [
            ('num', number_transformer, self.numbers), 
            ('categ',categ_transformer, self.categ)])
    
        #self.imputer = preprocessor.fit(self.dataset)
        a = preprocessor.fit_transform(self.dataset)
        b = preprocessor.fit(self.dataset)
        return b
        print('fit was performed')
    
    def get_col(self):
        return self.labels

In [ ]:
miss_imputer = Missing_Values(data_scaled_250)
my_imputer = miss_imputer.fit_imputer()
col_names = miss_imputer.get_col()

#New data frame

data_scaled_250_new = pd.DataFrame(my_imputer.transform(data_scaled_250), columns = col_names)
data_scaled_250_new.head()



Once we have dealt with the scaling, we can detect the outliers! As described above, there are different ways of for detecting outliers. The following outlier class has two options from which you can choose from, the so-called local outlier factor as well as the the chauvenets criterion that represents a distribution based method. Most importantly, it assumes the data to be normally distributed! You can check the data (for here now only visually) and see if it approximately normally distributed.

The following class will give you some options to play around with:
- visualizing a feature and plot it
- do a Q-Q plot 
- do a statistical test for testing if features are normally distributed. Here we will use the Shapiro-Wilk test.

The Shapiro Wilk test returns a so-called p-value. It assumes first that the data is normally distributed. If the p-value falls below a certain threshold, the so-called significance level, people say that there is evidence against the null hypothesis and will reject than the hypothesis. Normal threshold levels, alpha levels, are 0.01, 0.05 and 0.10. Check the data and compare the p-values with the alpha value that you decide on! The lower the alpha level, the 'more strict' you are! If you use the option 'answer', it will show you the answer based on the alpha level that you chose! The default value is 0.05 as it is commonly used in Statistics.

Play around with it and see if your data is normally distributed! 

In [ ]:
data_scaled_250_new.head()

In [ ]:
class visualizer_normality:
    def __init__(self, dataset, feature_list):
        self.features = feature_list
        self.length = len(feature_list)
        self.data = dataset
    def qq_plot(self):
        for i in range(0, self.length):
            axs[i] = qqplot(data = self.data[self.features[i]].values, line = 's')
            plt.title(f'QQ plot for {self.features[i]}')
            plt.tight_layout()
    def density_plot(self):
        fig, ax = plt.subplots(self.length, tight_layout = True)
        for i in range(0, self.length):
            sns.kdeplot(self.data[self.features[i]].values, bw_method=0.5, ax = ax[i])
            ax[i].title.set_text(f'Density plot for {self.features[i]}')
    def statistical_test(self, alpha = 0.05, answer = False):
        output_no_answer = dict()
        output_answer = dict()
        for i in range(0, self.length):
            p_value = shapiro(self.data[self.features[i]].values)[0]
            if p_value  < alpha:
                answer_test = 'The test suggests that the feature is not normally distributed'
            else:
                answer_test = 'The test suggests that the feature is normally distributed'
            output_answer[self.features[i]] = (p_value, answer_test)
            output_no_answer[self.features[i]] = (p_value)
        if answer == False:
            return output_no_answer
        else:
            return output_answer

Inspect the features of the dataset and check if they are normally distributed! This is important to know for you as some outlier detection methods assume normal distribution!

In [ ]:
#Input here what features you want to inspect! 
feature_list = ['acc_phone_x', 'acc_phone_y', 'acc_phone_z']

In [ ]:
my_visualizer = visualizer_normality(data_scaled_250_new, feature_list)

In [ ]:
my_visualizer.density_plot()
p = my_visualizer.statistical_test(answer= True)

Now that we are done with handling missing data and checking we can move on to remove outliers! To recall, there are two main categories of outlier detection:

- distance based
- distribution based

Here, we will provide you with the option to choose between a gaussian mixture model (assuming a Multinomial distribution) for the distribution based method, and the so-called local outlier factor as an example of a distance-based method. Try them out and see if the methods produce different results! 

Both models ask you to use a feature based on which it will detect an outlier. The Gaussian Mixture model will also ask you to input a threshold. This number should be an integer which represents the percentile. Normally, you should go for a low one such as 1,5, or maybe 10. The number of components is also a very crucial parameter, as it basically tells you how many normal distributions do we have! It will return a probability again similar to what we have seen before. 

In a next step, you can visualize the inlier and outlier points so you can check it visually if you did a good job!

After you have decided on a method to remove any outliers (or maybe you decided to keep all data), you can use the remove method to remove any outlier data.

In [ ]:
class Outlier_detection:
    def __init__(self, dataset):
        self.data = dataset
    
    def chauvenet_criterion(self, feature, c):
        mean = self.data[feature].mean()
        std = self.data[feature].std()
        N = len(self.data[feature].values)
        threshold_chauv = 1/(c*N)
        assigned_value = []
        for i in range(0, N):
            value = scipy.stats.norm(mean, std).cdf(self.data[feature][i])
            if (value < threshold_chauv) or ((1-value) < threshold_chauv):
                assigned_value.append(-1)
            else:
                assigned_value.append(1)
        self.data['Chauvenet Outlier'] = assigned_value
        
    def local_outlier_fit(self, number_of_obs):
        self.lof = LocalOutlierFactor(n_neighbors = number_of_obs)
        print('The local outlier factor has been fit!')
        self.data['LOF Outlier'] = self.lof.fit_predict(self.data)
        
            
    def gaussian_mixture_fit(self, feature, components, threshold):
        self.gm = GaussianMixture(n_components = components)
        gm_data = self.data[feature].values.reshape(-1,1)
        self.gm.fit(gm_data)
        self.data['GM score'] = self.gm.score_samples(gm_data)
        threshold = np.percentile(self.data['GM score'].values, threshold)
        self.data['GM Outlier'] = np.where(self.data['GM score'] < threshold, -1, 1)
        print('The Gaussian mixture model has predicted the labels!')
        

    def visualize_outlier(self, feature, method_used):
        if method_used == 'local_outlier':
            sns.scatterplot(x = self.data.index.values, y = self.data[feature], data = self.data, hue = 'LOF Outlier')
        elif method_used == "gaussian_mixture":
            sns.scatterplot(x = self.data.index.values, y = self.data[feature], data = self.data, hue = 'GM Outlier')
        elif method_used == 'chauvenet_criterion':
            sns.scatterplot(x = self.data.index.values, y = self.data[feature], data = self.data, hue = 'Chauvenet Outlier')
            
    def return_data(self):
        return self.data
    
    def return_data_outlier_removed(self, method):
        if method == 'local_outlier':
            return_data = self.data[self.data['LOF Outlier'] != -1].reset_index(drop = True)
            return return_data.drop(['LOF Outlier','GM Outlier','GM score','Chauvenet Outlier'], axis = 1)
        elif method == 'gaussian_mixture':
            return_data = self.data[self.data['GM Outlier'] != -1].reset_index(drop = True)
            return return_data.drop(['LOF Outlier','GM Outlier','GM score','Chauvenet Outlier'], axis = 1)
        elif method == 'chauvenet_criterion':
            return_data = self.data[self.data['Chauvenet Outlier'] != -1].reset_index(drop = True)
            return return_data.drop(['LOF Outlier','GM Outlier','GM score','Chauvenet Outlier'], axis = 1)
    
    def return_data_outlier_replaced(self, outlier_column, imputer):
        self.label = [col for col in self.data.columns if col.startswith('Active')]
        self.xdata = self.data.loc[:, ~self.data.columns.isin(self.label)]
        self.ydata = self.data.loc[:, self.label]
        xmean = self.xdata.groupby(outlier_column, as_index = False).agg('mean')
        xmean_values = xmean.loc[1, xmean.columns != outlier_column].values
        index_replace = list(self.xdata[self.xdata[outlier_column] == -1].index)
        self.xdata.loc[index_replace, ~self.xdata.columns.isin([outlier_column])] = xmean_values
        
        self.new_data = pd.concat([self.xdata, self.ydata], axis = 1)
        return self.new_data.drop(['LOF Outlier','GM Outlier','GM score','Chauvenet Outlier'], axis = 1)
            
    def return_outlier_mean_description(self, outlier_column):
        self.label = [col for col in self.data.columns if col.startswith('Active')]
        self.xdata = self.data.loc[:, ~self.data.columns.isin(self.label)]
        return self.xdata.groupby(outlier_column, as_index = False).mean()
            

In [ ]:
dataset_new = data_scaled_250_new.copy()

In [ ]:
outlier_det = Outlier_detection(dataset_new)
outlier_det.local_outlier_fit(20)
outlier_det.gaussian_mixture_fit('acc_phone_x',3,5)
outlier_det.chauvenet_criterion('acc_phone_x', 2)

Play around here! Visualize after you have fit the algorithms the outliers. Do you agree with the algorithm? Maybe you do not want to remove anything at all? That is also fine! It is always important to visualize and convinve yourself with the results of an algorithm. The following code visualizes the results of the LOF outlier and plots it for the 'acc_phone_x' value!

In [ ]:
outlier_det.visualize_outlier('acc_phone_x', 'local_outlier')

After we have visualized some of the results, we need to proceed and remove outliers. You can decide on one method or as I said before decide to not do anything! Also, when you remove outliers, you can either decide to remove them completely or replace it with a mean or median value. Again, think of the pros/cons of each of the two methods!


In [ ]:
dataset_new = outlier_det.return_data_outlier_replaced('LOF Outlier', 'mean')

In [ ]:
dataset_new.head()

## Handling noise - PCA decomposition 

For the principal component, the algorithm tries to reduce the number of features (here we have about 30 now even) by 'mapping'/aligning the features into a direction in which the variables vary the most. It is quite tough to imagine this, but the following video helps and explains the algorithm in more detail. **You do not need to watch it, but if you are curious and wanna get a better understanding, it might help!**

[The link to the Youtube video](https://www.youtube.com/watch?v=FgakZw6K1QQ&t=1126s)

Of course you should start from the start with the video. Watch the video though only if you have time! After all it is a Hackathon!

The following code implements the pca algorithm. One important consideration for you to consider is how many so-called principal components do you want to keep. For this reason, the plot 'explained variance' will help you figuring this out. **Decide in your group if you want to use the algorithm at all and if so, how many features you want to use!**.

An important aspect you need to be aware of is that you cannot intrepret the principal components any more. This is a big drawback of the method!

In [ ]:
#Important go through the methods step by step!

class pca_reduction:
    def __init__(self, data, features = None):
        self.features = features
        self.data = data
        self.numerical = [col for col in self.data.columns if not col.startswith('Active')] #Remove final label to be predicted
        self.ylabel = [col for col in self.data.columns if col.startswith('Active')]
        self.xdata_orig = self.data.loc[:, self.numerical]
        self.xdata = self.data.loc[:, self.numerical]
        self.ydata = self.data.loc[:, self.ylabel]
        self.pca = PCA()

    def fit_pca(self, data_to_fit = None):
        
        if self.features is None:
            self.pca.fit(self.xdata_orig)
        else:
            self.pca.fit(self.xdata_orig[features])
        self.components_pca = self.pca.components_.shape[1]
        self.explained_variance = self.pca.explained_variance_ratio_
        print('PCA was fit successfully to the features!')
        
        if data_to_fit is None:
            self.transformed_values = self.pca.transform(self.xdata_orig)
       
        else:
            self.xdata_fit = data_to_fit.loc[:, self.numerical]
            self.transformed_values = self.pca.transform(self.xdata_fit)
            
    def visualize_pca_components(self):
        
        self.components_plot = np.arange(1, self.components_pca+1,1)
        sns.set(style="whitegrid")
        ax = sns.barplot(x=self.components_plot, y=self.explained_variance)
        ax.set_title('Results PCA analysis')
        ax= ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
                    
    def return_pca_components(self, number_comp, data_to_fit = None, fit = False, attach_to_dataset = True):
        if fit == True and data_to_fit is not None:
            self.xdata = data_to_fit.loc[:, self.numerical]
            self.ydata = data_to_fit[self.ylabel]
            for comp in range(0, number_comp):
                self.xdata['pca_' +str(comp+1)] = self.transformed_values[:,comp]

        #elif attach_to_dataset == True:
        else:
            #And add the new ones:
            for comp in range(0, number_comp):
                self.xdata['pca_' +str(comp+1)] = self.transformed_values[:,comp]
            
        return pd.concat([self.xdata, self.ydata], axis = 1).dropna()

In [ ]:
dataset_NEW = dataset_new.copy()

In [ ]:
abc = dataset_new.copy()

In [ ]:
data_pca = dataset_new.copy()

In [ ]:
my_pca = pca_reduction(dataset_NEW)
my_pca.fit_pca()
my_pca.visualize_pca_components()

In [ ]:
data_tada = my_pca.return_pca_components(1)

In [ ]:
data_tada.head()

# Part 2: Feature engineering 

In this section, we will consider a few options for you to add new features. This is in practice an important field, but one also needs to be creative and/or use domain knowledge (that is, one has a certain expertise in the field) when defining new variables. Here we will consider a very basic option for you. To recall, each observation in the dataset represents an observation in time. One possible way to generate a new feature is to take the mean (or median, max, min) over a specific time window. This could be for example 5. This means I would summarize the information of 5 time observations in one new feature! 
The functions that you can insert are 'mean', 'max', 'min', 'std'.

Check the following code out and see what features you want to add and also what aggregate function and time window (this is quite important. Maybe think also about the granularity of the dataset!).

One way to see if you should go for a specific time window is to plot values and see how 'smooth' the line is for a certain window size. Try it!

For instance, go for window size = 20 (which means 5 seconds, as 4 samples represent a datapoint). Alternatively, you can go for 120, which is 30 seconds! However try out several things!

In [ ]:
class rolling_window_aggregator:
    def aggregate(self,data, window, aggregation_function):
        self.data = data
        #Keep only numerical data
        numerical = [col for col in self.data.columns if not col.startswith('Active')]
        new_col = [col + aggregation_function for col in numerical]
        data_num = self.data.loc[:, numerical] 
        if aggregation_function == 'mean':
            data_num.rolling(window).mean()
            data_num.columns = new_col
            return data_num
        
        elif aggregation_function == 'max':
            data_num.rolling(window).max()
            data_num.columns = new_col
            return data_num
        
        elif aggregation_function == 'min':
            data_num.rolling(window).min()
            data_num.columns = new_col
            return data_num
            
        elif aggregation_function == 'median':
            data_num.rolling(window).median()
            data_num.columns = new_col
            return data_num
        
        elif aggregation_function == 'std':
            data_num.rolling(window).std()
            data_num.columns = new_col
            return data_num
        
    def concat_data(self, dataset_original, dataset_2):
        self.data_new = pd.concat([dataset_original, dataset_2], axis = 1)
        return self.data_new
    
    def visualize_plot(self, feature_to_plot):
        plt.plot(np.arange(0, len(self.data_new[feature_to_plot])), self.data_new[feature_to_plot])

To include more than one aggregation feature in your original dataset, use the method 'aggregate' as many times as you want to create new features. For instance, use the 'aggregate method' twice for mean and std. deviation. Then, you can start to put your data together. This is done by using 'concat_data'. 

**Important** 
First concate data data together that only has the new features, i.e. those were the outputs of the method 'agggregate'. In a last step, concate your original data to the just concated dataframe.

Below you find an example of how things could work out!

In [ ]:
testy = data_tada.copy()

In [ ]:
class_roll = rolling_window_aggregator()
testy_mean = class_roll.aggregate(testy, 120, 'mean')
testy_std = class_roll.aggregate(testy, 120, 'std')

In [ ]:
new1 = class_roll.concat_data(testy_std, testy_mean)

In [ ]:
NEW_dataset = class_roll.concat_data(testy, new1)

In [ ]:
NEW_dataset.columns

## Dimensionality reduction / Feature selection (OPTIONAL)

**Important note**:
- Feature selection as it is illustrated below should be normally part of the cross-validation used below (when we train the models and use GridSearchCV). Hence, using feature selection before using cross-validation is not suggested and therefore the code below is more for you to play around and for educational purposes!

Given that we have now a lot of features, a lot of people use techniques to reduce the numbers of features in a smart way. We will have a look at two methods, i.e.

- Principal components
- Feature selection via Forward selection

We have seen the principal component analysis algorithm already before, so here we will focus on the Feature selection method. Here, the idea is that we will figure out step-by-step which features are more useful from a given set of features. We will first check among all features which is the one that will help us the most with the classification task at hand. After we have found one, we will continue our search with adding a second one (to the one that we have already found). 

In the following list, you can add the variables that you want to select from. This will be the input of the algorithm. 

In [ ]:
#Writting down the variables for the feature selection process
#Write here the features that you want to input in the algorithm
feature_list = ['acc_phone_x', 'acc_phone_y', 'acc_phone_z']

#Alternatively, you could just use the following command to get all features if you want to test all
feature_list = [col for col in NEW_dataset.columns if col != 'Active']
len(feature_list)

In [ ]:
class feature_selection_algorithm:
    def __init__(self, data, feature_list):
        self.data = data
        self.feature_list = feature_list
        self.ydata = data['Active'].values
        self.xdata = self.data.loc[:,feature_list]
    def fit_selection(self, number_of_features_to_select):
        self.selector = SelectKBest(f_classif,k=number_of_features_to_select).fit(self.xdata,self.ydata)
    def transform(self):
        self.support = self.selector.get_support()
        self.columns = [key for key, value in dict(zip(self.feature_list, self.support)).items() if value == True ]
        self.dfydata = pd.DataFrame(self.ydata, columns = ['Active'])
        self.dfxdata = pd.DataFrame(self.selector.transform(self.xdata), columns = self.columns)
        return pd.concat([self.dfxdata, self.dfydata], axis = 1)

In [ ]:
ft_selector = feature_selection_algorithm(NEW_dataset, feature_list)
ft_selector.fit_selection(10)

In [ ]:
final_data = ft_selector.transform()

In [ ]:
final_data

# Part 3: Training your model

After the final steps, we can finally start to begin to train our model and check our predictions! Yet, there is an important first step that we need to do before starting to train the model. 

## Handling unbalanced dataset 

In [ ]:
NEW_dataset.head()

In [ ]:
#Check the proportion of the classes that we want to estimate.
sns.histplot(NEW_dataset['Active'])

As we can see, there is a huge so-called imbalanced in the dataset. There are way more instances of User being inactive than active! If an algorithm would now always predict 0, i.e. inactive, it would be correct in about 80% of the cases already. To prevent this from happening, there are several solutions:

**Sampling approaches**
- upsampling the minority class
- downsampling the majority class
- create an artificial data set using the so-called SMOTE algorithm

It is important to note that all sampling approaches are should be only applied to the training data set! So that is, we had split our data set into a train and test set, we would have to apply sampling approaches only to the training data set and **NOT** the test set!

**Question for you**:
What are main disadvantages of the upsampling and the downsampling approach? 

**Performance metric approach**
- We can use a different evaluation metric. Instead of saying, how many times are we correct, we can also use a more balanced approach which also considers how often we were incorrect. For example the F1 score is a more balanced approach. Alternatively, there is the ROC-AUC measure. This is the one that we will use. The closer the score to 1, the better! A score of 0.5 would mean that we are at random guess, so the model is as good as flipping a coin!

**Model approach**
- There are specific machine learning models that are more suited when dealing with imbalanced data. Those methods will focus on mistakes and will iteratively try to get them correct over the long run. The class of models that are in this category are called boosting algorithms.


## Play around 

In the following code, you can choose one of the sampling approaches. Just play around and see what you want to use and combine it with some models.

In [ ]:
NEW_dataset

In [ ]:
class imbalance_sampler:
    def __init__(self, data):
        self.data = data
        self.ydata = data['Active'].values
        self.xdata = self.data.drop(['Active'], axis=1)
    
    def smote(self):
        self.sm = SMOTE(random_state=12)
        self.x_train_sm, self.y_train_sm = self.sm.fit_resample(self.xdata, self.ydata)
        return self.x_train_sm, self.y_train_sm
    
    def downsampling(self):
        self.downsampler = RandomUnderSampler(random_state=0)
        self.x_train_ds, self.y_train_ds = self.downsampler.fit_resample(self.xdata, self.ydata)
        return self.x_train_ds, self.y_train_ds
    
    def oversampling(self):
        self.oversampler = RandomOverSampler(random_state=0)
        self.x_train_os, self.y_train_os = self.oversampler.fit_resample(self.xdata, self.ydata)
        return self.x_train_os, self.y_train_os        

In [ ]:
NEW_dataset

In [ ]:
im = imbalance_sampler(NEW_dataset)
sm_data = im.smote()
ds_data = im.downsampling()
os_data = im.oversampling()

The following class will implement different 'classical' models for you that are not particularly suited for imbalanced datasets. Yet, if you have done the previous step, you already have some adjusted data set!

In the following, we will consider methods that are more suitable for imbalanced data sets. Hence we will consider three models (do no worry about the ins and outs of the models). You can google and read upon them if you want to get a further understanding. This workshop is just here for you to learn and see what is out there! You can train them by including your dataset that you have obtained before! In particular, you have two options:

- Support Vector Machine (SVM)
- Logistic Regression (LR)

The SVM model performs often times more superior compared to the LR, yet the latter has is better when it comes to explaining results! Yet, it takes way more computational time. Please be aware of this before trying it out!

You can also try to use :

- AdaBoost Classifier (ADA)

which is a boosting algorithm!

In [ ]:
class ml_model():
    def __init__(self, data, sampling_approach_before = True):
        if sampling_approach_before == True:
            self.xdata = data[0]
            self.ydata = pd.DataFrame(data[1], columns = ['Activity'])
            self.ydata = data[1]
        else:
            self.ydata = data['Active'] 
            self.xdata = data.drop(['Active'], axis = 1) 
    def fit_LR(self):
        #return self.xdata
        self.myLR = LogisticRegression(random_state = 0)
        self.my_grid_values_lr = {'penalty': ['l2','none'], 'solver': ['sag'], 'max_iter': [100]}
        print('Please wait the LR best model is going to be fit!')
        self.grid_myLR = GridSearchCV(self.myLR, param_grid = self.my_grid_values_lr, cv = 3, n_jobs = 5, scoring = 'roc_auc', error_score = 'raise')
        self.grid_myLR.fit(self.xdata, self.ydata)
        print('Best penalty parameter : '+ str(self.grid_myLR.best_estimator_.penalty))
        print('Best ROC AUC score : '+ str(self.grid_myLR.best_score_))
        
    def fit_SVM(self):
        self.mySVM = SVC(random_state = 0)
        self.my_grid_values_svm = {'kernel': ['linear'], 'C': [0, 1]}
        print('Please wait the best SVM model is going to be fit!')
        self.grid_mySVM = GridSearchCV(self.mySVM, param_grid = self.my_grid_values_svm, cv = 3, n_jobs = 5, scoring = 'roc_auc')
        self.grid_mySVM.fit(self.xdata, self.ydata)
        print('Best C parameter : '+ str(self.grid_mySVM.best_estimator_.C))
        print('Best ROC AUC score : '+ str(self.grid_mySVM.best_score_))
    
    def fit_ADA(self):
        self.myADA = AdaBoostClassifier(random_state = 0)
        self.my_grid_values_ada = {'n_estimators': [5,10,50,100]}
        print('Please wait the best ADA model is going to be fit!')
        self.grid_myADA = GridSearchCV(self.myADA, param_grid = self.my_grid_values_ada, cv = 3, n_jobs = 5, scoring = 'roc_auc')
        self.grid_myADA.fit(self.xdata, self.ydata)
        print('Best number of estimators parameter : '+ str(self.grid_myADA.best_estimator_.n_estimators))
        print('Best ROC AUC score : '+ str(self.grid_myADA.best_score_))
    
    def prediction_LR(self, xtest):
        return self.grid_myLR.predict(xtest)
    
    def prediction_SVM(self, xtest):
        return self.grid_mySVM.predict(xtest)
   
    def prediction_ADA(self, xtest):
        return self.grid_myADA.predict(xtest)
                                                                            
                                                                

In [ ]:
vanilla_ada = ml_model(ds_data, sampling_approach_before = True)

In [ ]:
vanilla_ada.fit_ADA()

Also try to fit a logistic regression model following this exanple. Also you should try to fit a SVM model. Which one performs best?

# Your turn! Train your own algorithm with your own data!

Now it is time for you to go through all steps from before and use your own test data that you have collected!

## Importing the training data set

In the first step, you need to import the data that you collected via the software. For this, record the data via the 'Sensor Logger' app and export it. Tip: Name the data set right away as being active or not. To recall: active means either running or walking, whereas 'Sitting', 'WashingHands', 'Standing', 'Driving', 'Eating' were classified as inactive. A value of 1 for the feature was assigned if the user was active, and zero otherwise.

You should export the dataset as ziped CSV! Then save it in a place of your choice and unzip the data file. As a result, you will get a folder (named the way you named your dataset). In that folder, you will find then four datasets ('Accelerometer', 'Gyroscope', 'Light' and 'Magnetometer'). Also, there is a metadata file. 

Do one activity for certain amount of seconds. Then save that individual data file and save this. Do each data set step by step, i.e. do not do several activities at the same time. Do for instance 10 seconds of walking. Save and export the data file. Convert that data file with the function below.

## Converting the raw data into aggregate data 

To convert the data into the format we need, we need to use the following function.

**Important**: Change the paths dataset_path, result,path, result_fname according to how you have saved the data!

In [ ]:
DATASET_PATH = files.upload()

In [ ]:
## THIS IS THE CODE TO RUN TO CREATE AND CONVERT YOUR OWN DATASET!

#DATASET_PATH = Path('./my_own_data/walking')
RESULT_PATH = Path('./intermediate_datafiles/')

# CHANGE THE RESULT DATA FILE NAME!
RESULT_FNAME = 'data_own.csv'

# Set a granularity (the discrete step size of our time series data). We'll use a course-grained granularity of one
# instance per minute, and a fine-grained one with four instances per second.
GRANULARITIES = [250]

# Do not change this
use_c = ['acc_phone_x','acc_phone_y','acc_phone_z','gyr_phone_x','gyr_phone_y','gyr_phone_z','light_phone_lux','mag_phone_x','mag_phone_y','mag_phone_z','Active']
# Add your label based on if you were active = 1 or not 0

label = 1

# We can call Path.mkdir(exist_ok=True) to make any required directories if they don't already exist.
#[path.mkdir(exist_ok=True, parents=True) for path in [DATASET_PATH, RESULT_PATH]]


datasets = []
for milliseconds_per_instance in GRANULARITIES:
    print(f'Creating numerical datasets from files in {DATASET_PATH} using granularity {milliseconds_per_instance}.')

    # Create an initial dataset object with the base directory for our data and a granularity
    dataset = CreateDataset(DATASET_PATH, milliseconds_per_instance)

    # Add the selected measurements to it.

    # We add the accelerometer data (continuous numerical measurements) of the phone and the smartwatch
    # and aggregate the values per timestep by averaging the values
    dataset.add_numerical_dataset('Accelerometer.csv', 'time', ['x','y','z'], 'avg', 'acc_phone_')

    # We add the gyroscope data (continuous numerical measurements) of the phone and the smartwatch
    # and aggregate the values per timestep by averaging the values
    dataset.add_numerical_dataset('Gyroscope.csv', 'time', ['x','y','z'], 'avg', 'gyr_phone_')


    # We add the labels provided by the users. These are categorical events that might overlap. We add them
    # as binary attributes (i.e. add a one to the attribute representing the specific value for the label if it
    # occurs within an interval).
    #dataset.add_event_dataset('labels.csv', 'label_start', 'label_end', 'label', 'binary')

    # We add the amount of light sensed by the phone (continuous numerical measurements) and aggregate by averaging
    dataset.add_numerical_dataset('Light.csv', 'time', ['lux'], 'avg', 'light_phone_')

    # We add the magnetometer data (continuous numerical measurements) of the phone and the smartwatch
    # and aggregate the values per timestep by averaging the values
    dataset.add_numerical_dataset('Magnetometer.csv', 'time', ['x','y','z'], 'avg', 'mag_phone_')


    # Get the resulting pandas data table
    dataset = dataset.data_table

    datasets.append(copy.deepcopy(dataset))

    # If needed, we could save the various versions of the dataset we create in the loop with logical filenames:
    #dataset.to_csv(RESULT_PATH / f'data_preprocessed_own_data{milliseconds_per_instance}.csv')

# Make a table like the one shown in the book, comparing the two datasets produced.
#util.print_latex_table_statistics_two_datasets(datasets[0], datasets[1])

# Finally, store the last dataset we generated (250 ms).
# Add the label we have
dataset['Active'] = label
#dataset.to_csv(RESULT_PATH / RESULT_FNAME)
dataset_own = dataset.reset_index(drop = True)


Now you can have a look at your data! Import your dataset via the following command

In [ ]:
dataset_own.head()

## Working with several collected data sets (Optional)

If you want to collect several data sets, you can do that. Then please rerun the above code to create the dataset. Important is of course, that you then adjust the label value, so 0 in case this time you were not active. Also, make sure to change the name of the resulting datafile!

In [ ]:
class aggregating_datasets:
    
    def merge_dataset(self, dataset_1, dataset_2):
        print('The datasets were included and were merged!')
        return pd.concat([dataset_1, dataset_2])
        

In [ ]:
#Example code how you would merge two datasets. These are called dataset1 and dataset2 here! Uncomment them to make them work
#aggregator = aggregating_datasets()
#merged_data = aggregator.merge_dataset(dataset1, dataset2)

## Redo the preprocessing steps 

Now that you have merged the data, you need to redo the steps for preprocessing. In short, you need to standardization as well as the PCA and the imputing of missing variables again. Also the rolling window function needs to be redone.
Of course, if you used only a subset of features due to feaure selection you need to use only that subset as well!

In [ ]:
#Standardizer

data_own_scaled = pd.DataFrame(my_standardizer.transform(dataset_own), columns = col_names)

In [ ]:
#Missing data imputer

data_own_new = pd.DataFrame(my_imputer.transform(data_own_scaled), columns = col_names)

In [ ]:
data_own_new.columns

In [ ]:
#PCA Remember to take the same number of components as before!
my_pca.fit_pca(data_own_new)

data_own_NEW = my_pca.return_pca_components(1, data_own_new, True, True)

In [ ]:
data_own_NEW.head()

In [ ]:
#Rolling window
class_roll = rolling_window_aggregator()
data_own_mean = class_roll.aggregate(data_own_NEW, 4, 'mean') #Important check here window size!
data_own_std = class_roll.aggregate(data_own_NEW, 4, 'std')
new1 = class_roll.concat_data(data_own_mean, data_own_std)
NEW_dataset = class_roll.concat_data(data_own_NEW, new1)


Now you can finally split the dataset into Xtest and Ytest.

Now we can finally see how good our model performs on our data!

In [ ]:
#First split the dataset 

X_test = NEW_dataset.drop(['Active'], axis = 1)
Y_test = NEW_dataset[['Active']]

In [ ]:
Y_test.head()

We can make predictions given the models we had before. For example with the ADA Boost we can do the following:

In [ ]:
#ADA Boost get the predictions
y_pred = vanilla_ada.prediction_ADA(X_test)

Then, we can get the predictions via:

In [ ]:
print(accuracy_score(y_pred, Y_test))

---

&copy; 2022 - **VU Amsterdam**